# СИГНАЛ ПЛОХОЙ - ПЛОХО ОБРАБАТЫВАЕТСЯ
# ВОЗМОЖНО ПОМОГУТ ФИЛЬТРЫ 

# Помогли

In [12]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
from scipy.signal import resample, butter, filtfilt
from scipy.fft import fft, fftfreq

In [13]:
def funcBPFilter(ppg_signal, order, f1, f2, fs):
    fc = [f1, f2]
    b, a = butter(order, fc, btype='bandpass', analog=False, output='ba', fs=fs)
    signalBP = filtfilt(b, a, ppg_signal)
    return signalBP

In [14]:
age_group = {15:1.0,16:1.0,17:1.0,18:1.0,19:1.0,
             20:2.0,21:2.0,22:2.0,23:2.0,24:2.0,
             25:3.0,26:3.0,27:3.0,28:3.0,29:3.0,
             30:4.0,31:4.0,32:4.0,33:4.0,34:4.0,
             35:5.0,36:5.0,37:5.0,38:5.0,39:5.0,
             40:6.0,41:6.0,42:6.0,43:6.0,44:6.0,
             45:7.0,46:7.0,47:7.0,48:7.0,49:7.0,
             50:8.0,51:8.0,52:8.0,53:8.0,54:8.0,
             55:9.0,56:9.0,57:9.0,58:9.0,59:9.0,
             60:10.0,61:10.0,62:10.0,63:10.0,64:10.0,
             65:11.0,66:11.0,67:11.0,68:11.0,69:11.0,
             70:12.0,71:12.0,72:12.0,73:12.0,74:12.0,
             75:13.0,76:13.0,77:13.0,78:13.0,79:13.0,
             80:14.0,81:14.0,82:14.0,83:14.0,84:14.0,
             85:15.0,86:15.0,87:15.0,88:15.0,89:15.0,
             90:16.0,91:16.0,92:16.0,93:16.0,94:16.0,
             95:17.0,96:17.0,97:17.0,98:17.0,99:17.0,100:17.0,
             }

In [ ]:
subjects = pd.read_excel(r"D:/CK/subjects_metadata.xlsx",dtype = {"ID":str})
#subjects.dropna(inplace=True)
subjects = subjects[subjects['observations'].isna()]
subjects = subjects.reset_index()
subjects = subjects[["subjectcode","age"]]
subjects["age"] = subjects["age"].map(age_group)

In [16]:
ids = subjects["subjectcode"].values
age_groups = subjects["age"].values

In [ ]:
result = []
result_metadata = []

new_id = 1096

for id in ids:
    new_id += 1 
    try:
        df_ppg = pd.read_csv("D:/CK/lucilacapurro/PPG_Signal_Dataset/data/"+id+".csv")
        signal_ppg = df_ppg['PPG'].values
        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = signal_ppg.reshape(-1, 1).astype(np.float32)
        fs = 500 
        #print(len(signal[0]))
        five_minute_samples = 60 * 5 * fs

        signal = signal[:five_minute_samples].flatten()

        signal = funcBPFilter(signal, order=2, f1=0.5, f2=4.0, fs=fs)
        
        try:
            wd, m = hp.process(signal, fs)
            
            # Вычисляем частотные характеристики с помощью scipy
            n = len(signal)
            yf = fft(signal)
            xf = fftfreq(n, 1/fs)
            
            # Определяем границы для LF и HF
            lf_band = (0.04, 0.15)
            hf_band = (0.15, 0.4)
            
            # Вычисляем LF и HF
            lf_mask = (xf >= lf_band[0]) & (xf <= lf_band[1])
            hf_mask = (xf >= hf_band[0]) & (xf <= hf_band[1])
            
            lf = np.trapz(np.abs(yf[lf_mask])**2, xf[lf_mask])
            hf = np.trapz(np.abs(yf[hf_mask])**2, xf[hf_mask])
            lf_hf_ratio = lf / hf
            
            freq_features = {
                'lf': lf,
                'hf': hf,
                'lf_hf_ratio': lf_hf_ratio
            }

            idx = int(subjects[subjects["subjectcode"] == id].index.to_list()[0])
            age_group = age_groups[idx]

            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                'lf': freq_features['lf'],
                'hf': freq_features['hf'],
                'lf_hf_ratio': freq_features['lf_hf_ratio'],
                **m
            })

            print((new_id, id, age_group, freq_features['lf'], freq_features['hf'], freq_features['lf_hf_ratio']))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

(1097, 'S02', 1.0, 20138551146.930084, 1247535251262.3552, 0.016142671019958994)
Process end: S02
(1098, 'S03', 1.0, 2635688055.7230253, 60636740478.54042, 0.04346684922247438)
Process end: S03
(1099, 'S04', 1.0, 1360520839.8534014, 73150761281.25041, 0.01859886098276495)
Process end: S04
(1100, 'S05', 1.0, 79834451591.88548, 8544315827516.501, 0.009343574512400747)
Process end: S05
(1101, 'S06', 1.0, 583540418.0884587, 54488359312.84312, 0.010709451072624165)
Process end: S06
(1102, 'S07', 1.0, 2793710922.2089357, 83650037298.19888, 0.03339760521863017)
Process end: S07
(1103, 'S08', 1.0, 53249861094.82048, 480758394943.80994, 0.11076220749310933)
Process end: S08
(1104, 'S09', 1.0, 6827444516.693596, 111313543987.16437, 0.06133525420303636)
Process end: S09
(1105, 'S10', 1.0, 14428059893.11713, 773790322740.9648, 0.018645955459883786)
Process end: S10
(1106, 'S11', 1.0, 9083895427.903616, 226009727082.5801, 0.0401924976644236)
Process end: S11
(1107, 'S12', 2.0, 22774936870.994938, 3

In [ ]:
df_metadata = pd.DataFrame(result_metadata)
df_metadata.dropna(inplace=True)
df_metadata

,id,age_group,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1097,1.0,66.528681,901.866667,139.237383,99.014087,147.999289,0.815789,0.684211,109.0,104.401951,133.186849,43683.741807,0.783876,0.133333
1,1098,1.0,87.491410,685.781609,51.637758,18.079964,32.259425,0.554913,0.115607,34.0,22.808630,69.406116,4973.324812,0.328626,0.133333
2,1099,1.0,85.733017,699.847059,66.997455,51.641917,61.829504,0.568047,0.130178,32.0,43.718844,84.198819,11564.438156,0.519233,0.133333
3,1100,1.0,85.820597,699.132867,77.386166,76.555248,104.565914,0.765152,0.416667,44.0,73.934174,79.127325,18378.987331,0.934370,0.133333
4,1101,1.0,71.880441,834.719424,68.374343,36.878553,57.545412,0.678832,0.328467,42.0,40.660488,85.246936,10889.332014,0.476973,0.133333
5,1102,1.0,90.274373,664.640449,65.697083,53.676468,64.004971,0.534091,0.130682,38.0,45.258209,81.672066,11612.367872,0.554146,0.166667
6,1103,1.0,71.415046,840.159091,125.610951,83.204665,113.880099,0.707692,0.461538,80.0,78.388370,152.423695,37536.518707,0.514279,0.266667
7,1104,1.0,68.937551,870.352941,93.774725,32.705072,53.477583,0.755556,0.281481,70.0,37.810438,127.450770,15139.238762,0.296667,0.133333
8,1105,1.0,106.761566,562.000000,49.697196,38.460218,46.149766,0.421801,0.099526,28.0,32.630731,62.182855,6374.517617,0.524754,0.100000
9,1106,1.0,76.148706,787.931973,62.668294,19.759699,31.467155,0.454545,0.104895,48.0,22.245783,85.039842,5943.195424,0.261592,0.166667


In [ ]:
df_metadata.to_csv(r"D:/CK/dataset.csv",mode="a",index=False,header=False)